In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch.utils.data import random_split

In [57]:
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        height_after_pooling = (201 // 2) // 2
        width_after_pooling = (111 // 2) // 2
        self.fc1 = nn.Linear(64 * height_after_pooling * width_after_pooling, 512)  # Adjust the linear layer size based on your input
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        height_after_pooling = (201 // 2) // 2
        width_after_pooling = (111 // 2) // 2
        x = x.view(-1, 64 * height_after_pooling * width_after_pooling)  # Flatten the tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [58]:
transform = transforms.Compose([
    transforms.Resize((201, 111)), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])


In [59]:
full_dataset = datasets.ImageFolder(root='C:/D/Puyuan/UCSB/Capstone-Scripps/denoise_spectrogram', transform=transform)

train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size 
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=False)

In [60]:
model = SimpleCNN(num_classes=3)  # Replace with your actual number of classes
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [61]:
def train_model(model, criterion, optimizer, train_loader, num_epochs):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}')

num_epochs = 10  # Adjust as needed
train_model(model, criterion, optimizer, train_loader, num_epochs)


Epoch 1/10, Loss: 9.672756910324097
Epoch 2/10, Loss: 3.978116512298584
Epoch 3/10, Loss: 4.489539593458176
Epoch 4/10, Loss: 0.989498496055603
Epoch 5/10, Loss: 0.5928525626659393
Epoch 6/10, Loss: 0.543762817978859
Epoch 7/10, Loss: 0.43430860340595245
Epoch 8/10, Loss: 0.23933261260390282
Epoch 9/10, Loss: 0.20708658918738365
Epoch 10/10, Loss: 0.18962598592042923


In [66]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
import numpy as np

true_labels = []
model_predictions = []

with torch.no_grad(): 
    for images, labels in test_loader:
        outputs = model(images) 
        _, predicted = torch.max(outputs.data, 1)  
        true_labels.extend(labels.numpy())
        model_predictions.extend(predicted.numpy())

true_labels = np.array(true_labels)
model_predictions = np.array(model_predictions)

# Calculate the confusion matrix
cm = confusion_matrix(true_labels, model_predictions)
print(cm)

# Calculate precision, recall, and F1-score
print(classification_report(true_labels, model_predictions))



[[1 2]
 [1 6]]
              precision    recall  f1-score   support

           1       0.50      0.33      0.40         3
           2       0.75      0.86      0.80         7

    accuracy                           0.70        10
   macro avg       0.62      0.60      0.60        10
weighted avg       0.68      0.70      0.68        10

